## Week 10 - Kubernetes

by Jorge Abrego

### Bulding the image

Go to the `week_05-deployment/homework` folder and 
execute the following:


```bash
docker build -t zoomcamp-model:hw10 .
```

> **Note:** If you have troubles building the image, you can 
> use the image we built and published to docker hub:
> `docker pull svizor/zoomcamp-model:hw10`

### Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:hw10
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```python
{'get_credit': True, 'get_credit_probability': <value>}
```

Here `<value>` is the probability of getting a credit card. You need to choose the right one.

* 0.3269
* 0.5269
* 0.7269
* 0.9269

Now you can stop the container running in Docker.

In [1]:
!python ../week_05-deployment/homework/q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}


### Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

### Question 2

What's the version of `kind` that you have? 

Use `kind --version` to find out.

In [3]:
!kind --version

kind version 0.20.0


### Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info
```

### Question 3

Now let's test if everything works. Use `kubectl` to get the list of running services. 

What's `CLUSTER-IP` of the service that is already running there? 

In [4]:
!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   3m3s


### Question 4

To be able to use the docker image we previously created (`zoomcamp-model:hw10`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image`
* `kubectl apply`

Answer: 

```bash
kind load docker-image zoomcamp-model:hw10
```

### Question 5

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit
spec:
  selector:
    matchLabels:
      app: credit
  replicas: 1
  template:
    metadata:
      labels:
        app: credit
    spec:
      containers:
      - name: credit
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

Answer: 

Port is 9696, cause this is the pre-configured port in the image.

In [5]:
!kubectl get deployment

NAME     READY   UP-TO-DATE   AVAILABLE   AGE
credit   1/1     1            1           38s


### Question 6

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?

Apply this config file.

Answer:

We need to write `credit`, name of app deployed to our kubernetes cluster.

In [6]:
!kubectl get services

NAME         TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
credit       LoadBalancer   10.96.175.54   <pending>     80:30247/TCP   19s
kubernetes   ClusterIP      10.96.0.1      <none>        443/TCP        49m


### Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/<Service name> 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.

In [10]:
!python ../week_05-deployment/homework/q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}
